In [0]:
# DATABASE TO TEST

# -*- coding: utf-8 -*-
"""
author SparkByExamples.com
"""

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")]

df = spark.createDataFrame(data=data,schema=columns)

df.show(truncate=False)

+-----+------------+
Seqno|Name |
+-----+------------+
1 |john jones |
2 |tracey smith|
3 |amy sanders |
+-----+------------+

In [0]:
# DATABASE TO TEST

# importing module
import pyspark
 
# importing sparksession from pyspark.sql module
from pyspark.sql import SparkSession
 
# creating sparksession and giving an app name
spark = SparkSession.builder.appName('sparkdf').getOrCreate()
 
# list  of employee data
data = [["1", "sravan", "company 1"],
        ["2", "ojaswi", "company 1"],
        ["3", "rohith", "company 2"],
        ["4", "sridevi", "company 1"],
        ["5", "bobby", "company 1"]]
 
# specify column names
columns = ['ID', 'NAME', 'Company']
 
# creating a dataframe from the lists of data
dataframe = spark.createDataFrame(data, columns)
 
dataframe.show()

+---+-------+---------+
 ID| NAME| Company|
+---+-------+---------+
 1| sravan|company 1|
 2| ojaswi|company 1|
 3| rohith|company 2|
 4|sridevi|company 1|
 5| bobby|company 1|
+---+-------+---------+

In [0]:
# USING HASH IN PYSPARK

from pyspark.sql.functions import sha2

#df = dataframe.withColumn('sha-384', sha2(df['col1'],384))
#df = dataframe.withColumn('sha-512', sha2(df['col1'],512))
#df.select('sha-384').show(truncate=False)

df = dataframe.withColumn('NAME', sha2(dataframe['NAME'],512))
display(df)

ID,NAME,Company
1,61b5a277e806bb492ef009b13f832bc5f74dc2be33edf2033ba7021d61cd5eee36d23f1939d62a91a80926af72192a8411e57adae98faaf93829f26bb4f64e30,company 1
2,869e1dc3bbff06614c97dceebf6790f0125fbb21ccad08876d0f05f37bcd1509400a0cd80163c84d83d4ebc1dfd78221db2f2e9175247dc659b6d28add8bd2cb,company 1
3,4ff3f6cbd79d78214949012d357a5d28dc2a58f8243ef83564efe8a0c7e96a63e50fca5ee7833ee4b8ed4000b6e26b3e2a3ade11c01a7f0ec42c81a45a3d0d8d,company 2
4,8c661c6eaa161870d8a5782e8ec775264ad9c22e8f8c1d9c462f8147432c2c1201a6b638c20608219b9651c7aad3e7a9d32b3a2e4b7cb63d619ffad65fcfb103,company 1
5,bd21f705a4f32e747b86322971ff259c7295f3337fc8d62e0fceb94d35fa10ce4792e514fb4c6fae3b28b20cbbb74538746fc104f2ab0ef17ea8d1870484b926,company 1


In [0]:
# Applying convertUDF

def convertCase(str):
    resStr=""
    arr = str.split(" ")
    for x in arr:
       resStr= resStr + x[0:1].upper() + x[1:len(x)] + "dadada "
    return resStr
  
convertUDF = udf(lambda z: convertCase(z))

df.select(col("Seqno"), convertUDF(col("Name")).alias("Name") ).show(truncate=False) 

+-----+-------------------------+
Seqno|Name |
+-----+-------------------------+
1 |Johndadada Jonesdadada |
2 |Traceydadada Smithdadada |
3 |Amydadada Sandersdadada |
+-----+-------------------------+

In [0]:
df = sc.parallelize([('s', 'S', 'NS'), ('?', '?', '?')]).toDF(['a', 'b', 'c'])
df.display()

a,b,c
s,S,NS
?,?,?


In [0]:
@udf
def return_string(a, b, c):
    if a == 's' and b == 'S' and c == 's':
        return 'S'
    if a == 's' and b == 'NS' and c == 's':
        return 'S'
    if a == 's' and b == 'S' and c == 'NS':
        return 'NS'

df.withColumn('result', return_string('a', 'b', 'c')).show()
## +---+---+---+------+
## |  a|  b|  c|result|
## +---+---+---+------+
## |  s|  S| NS|    NS|
## |  ?|  ?|  ?|  null|
## +---+---+---+------+

+---+---+---+------+
 a| b| c|result|
+---+---+---+------+
 s| S| NS| NS|
 ?| ?| ?| null|
+---+---+---+------+

In [0]:
# -*- coding: utf-8 -*-
"""
author SparkByExamples.com
"""

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")]

df = spark.createDataFrame(data=data,schema=columns)

df.show(truncate=False)

def convertCase(str):
    resStr=""
    arr = str.split(" ")
    for x in arr:
       resStr= resStr + x[0:1].upper() + x[1:len(x)] + " "
    return resStr 

""" Converting function to UDF """
convertUDF = udf(lambda z: convertCase(z))

df.select(col("Seqno"), \
    convertUDF(col("Name")).alias("Name") ) \
.show(truncate=False)


@udf(returnType=StringType()) 
def upperCase(str):
    return str.upper()

upperCaseUDF = udf(lambda z:upperCase(z),StringType())    

df.withColumn("Cureated Name", upperCase(col("Name"))) \
.show(truncate=False)

""" Using UDF on SQL """
spark.udf.register("convertUDF", convertCase,StringType())
df.createOrReplaceTempView("NAME_TABLE")
spark.sql("select Seqno, convertUDF(Name) as Name from NAME_TABLE") \
     .show(truncate=False)
     
spark.sql("select Seqno, convertUDF(Name) as Name from NAME_TABLE " + \
          "where Name is not null and convertUDF(Name) like '%John%'") \
     .show(truncate=False)  
     
""" null check """

columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders"),
    ('4',None)]

df2 = spark.createDataFrame(data=data,schema=columns)
df2.show(truncate=False)
df2.createOrReplaceTempView("NAME_TABLE2")
    
spark.udf.register("_nullsafeUDF", lambda str: convertCase(str) if not str is None else "" , StringType())

spark.sql("select _nullsafeUDF(Name) from NAME_TABLE2") \
     .show(truncate=False)

spark.sql("select Seqno, _nullsafeUDF(Name) as Name from NAME_TABLE2 " + \
          " where Name is not null and _nullsafeUDF(Name) like '%John%'") \
     .show(truncate=False)  


In [0]:

# importing module
import pyspark
 
# importing sparksession from pyspark.sql module
from pyspark.sql import SparkSession
 
# creating sparksession and giving an app name
spark = SparkSession.builder.appName('sparkdf').getOrCreate()
 
# list  of employee data
data = [["1", "sravan", "company 1"],
        ["2", "ojaswi", "company 1"],
        ["3", "rohith", "company 2"],
        ["4", "sridevi", "company 1"],
        ["5", "bobby", "company 1"]]
 
# specify column names
columns = ['ID', 'NAME', 'Company']
 
# creating a dataframe from the lists of data
dataframe = spark.createDataFrame(data, columns)
 
dataframe.show()

+---+-------+---------+
 ID| NAME| Company|
+---+-------+---------+
 1| sravan|company 1|
 2| ojaswi|company 1|
 3| rohith|company 2|
 4|sridevi|company 1|
 5| bobby|company 1|
+---+-------+---------+

In [0]:
import pandas as pd
from pyspark.sql.functions import pandas_udf
from pyspark.sql import Window

df = spark.createDataFrame(
    [(1, 1.0), (1, 2.0), (2, 3.0), (2, 5.0), (2, 10.0)],
    ("id", "v"))


df.show()

+---+----+
 id| v|
+---+----+
 1| 1.0|
 1| 2.0|
 2| 3.0|
 2| 5.0|
 2|10.0|
+---+----+

In [0]:
from pyspark.sql.functions import pandas_udf
# Declare the function and create the UDF
@pandas_udf("double")

#def mean_udf(v):
#    return v.mean()
def mean_udf(v: pd.Series) -> float:
    return v.mean()

In [0]:
df.select(mean_udf(df['v'])).show()

+-----------+
mean_udf(v)|
+-----------+
 4.2|
+-----------+

In [0]:
from pyspark.sql.functions import pandas_udf, PandasUDFType  
# Declare the function and create the UDF

@pandas_udf("float")
#def mean_udf(v):
#    return v.mean()
def mean_udf(v: pd.Series) -> float:
    return v.mean()

In [0]:
df.select(mean_udf(df['v'])).show()

+-----------+
mean_udf(v)|
+-----------+
 4.2|
+-----------+